In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import LabelEncoder
from keras.layers import LSTM,Dense,Dropout, Reshape, Conv1D, GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.models import Sequential
from keras.layers import Embedding,SimpleRNN
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

from tensorflow.keras.optimizers import Adam

from gensim.models import Word2Vec
import gensim

import ast
import nltk

In [2]:
from google.cloud import storage
import os
import io
client = storage.Client()
bucket = client.get_bucket('nlp_final_data')

blob = bucket.blob('songs_combined_ngrams.csv')
content = blob.download_as_string()

df = pd.read_csv(io.BytesIO(content))

In [3]:
df['up_to_bigrams'] = df['up_to_bigrams'].apply(lambda x: x.strip('[]').replace('\'', '').split(', '))

In [4]:
X = df['up_to_bigrams']
y = df['tag']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=42)

In [40]:
y_train[0]

'rap'

In [8]:
# Train the Word2Vec model
w2v_model = Word2Vec(X_train, vector_size=1000, window=5, min_count=5, workers=4)

In [9]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1


In [10]:
# Pad the sequences to a fixed length
max_length = 1000
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

In [11]:
# Create a weight matrix for the embedding layer
embedding_matrix = np.zeros((vocab_size, 1000))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

MemoryError: Unable to allocate 41.4 GiB for an array with shape (5557820, 1000) and data type float64

In [ ]:
def vectorize(words):
    words_vecs = [model.wv[word] for word in words if word in model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train = np.array([vectorize(sentence) for sentence in X_train])
X_test = np.array([vectorize(sentence) for sentence in X_test])

In [ ]:
encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

num_classes = np.max(y_train) + 1
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [ ]:
len(X_train[4])

In [ ]:
# Define the CNN model
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes,activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
model3.save('model_w2v_bigrams.h5')

In [ ]:
blob_name = 'model_w2v_bigrams.h5'
client = storage.Client()

blob = bucket.blob(blob_name)
blob.upload_from_filename(blob_name)